In [1]:
import os
import pandas as pd

# RATE_SPEECHES_FILE = "joined.parquet.gzip"
# assert os.path.exists(RATE_SPEECHES_FILE), f"file not present: {RATE_SPEECHES_FILE}"

# rate_speeches = pd.read_parquet(RATE_SPEECHES_FILE)


In [2]:
import nltk
nltk.download('punkt_tab')

from nltk import word_tokenize
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
# %%time

# def rate_speeches_text_length():
#   '''
#   reload rate_speeches or update it with text lengths
#   '''
#   global rate_speeches
#   RATE_SPEECHES_TL_FILE = 'rate_speeches.text_length.parquet.gzip'

#   if os.path.exists(RATE_SPEECHES_TL_FILE):
#     print(f"loading {RATE_SPEECHES_TL_FILE}...")
#     rate_speeches = pd.read_parquet(RATE_SPEECHES_TL_FILE)
#   else:
#     rate_speeches['text_length'] = rate_speeches['extracted_text'].apply(lambda x: len(word_tokenize(x)))
#     rate_speeches.to_parquet(RATE_SPEECHES_TL_FILE, compression='gzip')

# rate_speeches_text_length();

# print(rate_speeches['text_length'].describe())
# rate_speeches['extracted_text'][0]


In [26]:

def rate_speeches_sent_tokenize():
  '''
  reload rate_speeches or update it with `sent_tokenize`
  '''
  global rate_speeches
  RATE_SPEECHES_TOKENIZED_FILE = 'rate_speeches.sent_tokenize.parquet.gzip'

  if os.path.exists(RATE_SPEECHES_TOKENIZED_FILE):
    print(f"loading {RATE_SPEECHES_TOKENIZED_FILE}...")
    rate_speeches = pd.read_parquet(RATE_SPEECHES_TOKENIZED_FILE)
  else:
    rate_speeches['extracted_text'] = rate_speeches['extracted_text'].apply(sent_tokenize)
    rate_speeches.to_parquet(RATE_SPEECHES_TOKENIZED_FILE, compression='gzip')

rate_speeches_sent_tokenize()


loading rate_speeches.sent_tokenize.parquet.gzip...


In [5]:
import os
import nltk
import gensim
import numpy as np
import unicodedata

from itertools import groupby
from unicodedata import category as unicat

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.chunk import tree2conlltags
from nltk.probability import FreqDist
from nltk.chunk.regexp import RegexpParser
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.utils import pad_sequences


class KeyphraseExtractor(BaseEstimator, TransformerMixin):
    """
    Extract adverbial and adjective phrases, and transform
    documents into lists of these keyphrases, with a total
    keyphrase lexicon limited by the nfeatures parameter
    and a document length limited/padded to doclen
    """
    def __init__(self, nfeatures: int, doclen: int):
        self.grammar = r'KT: {(<RB.> <JJ.*>|<VB.*>|<RB.*>)|(<JJ> <NN.*>)}'
        # self.grammar = r'KT: {(<RB.*> <VB.>|<RB.>|<JJ.> <NN.*>)}'
        # self.grammar = r'KT: {<RB.>|<JJ.>}'
        self.chunker = RegexpParser(self.grammar)
        self.nfeatures = nfeatures
        self.doclen = doclen
        self._curr = 0

    def normalize(self, sent):
        """
        Removes punctuation from a tokenized/tagged sentence and
        lowercases words.
        """
        is_punct = lambda word: all(unicat(c).startswith('P') for c in word)
        sent = filter(lambda t: not is_punct(t[0]), sent)
        sent = map(lambda t: (t[0].lower(), t[1]), sent)
        return list(sent)

    def extract_candidate_phrases(self, sents, call_no):
        """
        For a document, parse sentences using our chunker created by
        our grammar, converting the parse tree into a tagged sequence.
        Extract phrases, rejoin with a space, and yield the document
        represented as a list of it's keyphrases.
        """
        print(f">> \t [{call_no}] extract_candidate_phrases...")

        for i, sent in enumerate(sents):
            tokens = word_tokenize(sent)
            pos_tags = pos_tag(tokens)
            normalized = self.normalize(pos_tags)
            # print(f"[{self._curr}] sent #{i}: {sent}")
            # print(f"[{self._curr}] \t {normalized}")

            chunks = tree2conlltags(self.chunker.parse(normalized))
            if not chunks or all(chunk[-1] == 'O' for chunk in chunks):
                #print(f"No valid chunks found in sentence: {sent}")
                continue

            phrases = [
                " ".join(word for word, pos, chunk in group).lower()
                for key, group in groupby(
                    chunks, lambda term: term[-1] != 'O'
                ) if key
            ]
            for phrase in phrases:
                yield phrase

    def fit(self, documents, y=None):
        return self

    def get_lexicon(self, keydocs):
        """
        Build a lexicon of size nfeatures
        """
        keyphrases = [keyphrase for doc in keydocs for keyphrase in doc]
        print("Keyphrases:", keyphrases[:5])
        fdist = FreqDist(keyphrases)
        counts = fdist.most_common(self.nfeatures)
        print("Frequency counts:", counts[:5])
        lexicon = [phrase for phrase, count in counts]
        return {phrase: idx+1 for idx, phrase in enumerate(lexicon)}

    def clip(self, keydoc, lexicon):
        """
        Remove keyphrases from documents that aren't in the lexicon
        """
        return [lexicon[keyphrase] for keyphrase in keydoc
            if keyphrase in lexicon.keys()]

    def transform(self, documents):
      self._curr += 1
      print(f">> [{self._curr}] KeyphraseExtractor.transform: {len(documents)}...")
      docs = [list(self.extract_candidate_phrases(doc, i)) for i, doc in enumerate(documents)]
      lexicon = self.get_lexicon(docs)
      clipped = [list(self.clip(doc, lexicon)) for doc in docs]
      return pad_sequences(clipped, maxlen=self.doclen)

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def data_set(view):
  X = view['extracted_text']
  display(X.describe())

  # view = rate_speeches[:doclen]
  labels = view["Direction"].values
  label_mapping = {label: idx for idx, label in enumerate(sorted(set(labels)))}
  encoded_labels = np.array([label_mapping[label] for label in labels])

  # convert to one-hot encoding
  num_classes = len(label_mapping)
  print(f"num_classes: {num_classes}")
  y = to_categorical(encoded_labels, num_classes=num_classes)
  return X, y, num_classes


In [7]:
!pip install scikeras

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense, Input
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

def create_lstm_model(vocab_size: int, input_length: int, num_classes: int):
    model = Sequential([
        Input(shape=(input_length,), name="input_layer"),  # explicit input layer
        Embedding(input_dim=vocab_size, output_dim=128, name="embedding_layer"),
        LSTM(128, return_sequences=True, name="lstm_layer_1"),
        Dropout(0.2, name="dropout_layer_1"),
        LSTM(64, name="lstm_layer_2"),
        Dropout(0.2, name="dropout_layer_2"),
        Dense(num_classes, activation="softmax", name="output_layer")
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [9]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [10]:

# X, y, num_classes = data_set(rate_speeches[:60])

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# keyphrase_extractor = KeyphraseExtractor(nfeatures=10000, doclen=100)
# X_train = keyphrase_extractor.fit_transform(X_train)
# X_val = keyphrase_extractor.transform(X_val)


In [11]:

# model = create_lstm_model(vocab_size=10000, input_length=100, num_classes=num_classes);

# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=10,
#     batch_size=32
# )


In [12]:

# loss, accuracy = model.evaluate(X_val, y_val)
# print(f"Validation Accuracy: {accuracy:.2f}")


In [13]:
class DebugTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        print("Data Shape Before LSTM:", X.shape)
        # print("Sample Data Before LSTM:", X[0])  # print a sample
        return X


In [14]:
# from sklearn.model_selection import cross_val_score, StratifiedKFold
# from joblib import Memory

# X, y, num_classes = data_set(rate_speeches)


In [15]:
# keyphrase_extractor = KeyphraseExtractor(nfeatures=10000, doclen=100)
# X_transformed = keyphrase_extractor.fit_transform(X)


In [16]:
# np.save("X_transformed.npy", X_transformed)


In [17]:
# np.save("y.npy", y)


In [18]:
X_transformed = np.load("X_transformed.npy")
y = np.load("y.npy")

display(X_transformed.shape)
display(y.shape)

(3434, 100)

(3434, 3)

In [19]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from joblib import Memory

# set up caching directory
memory = Memory(location="cache_directory", verbose=0)

pipeline = Pipeline([
    # ("keyphrase_extractor", # don't do this!, pre-compute this step instead!!!
    #   KeyphraseExtractor(nfeatures=10000, doclen=100)),
    ('debug', DebugTransformer()),
    ("lstm_classifier",
      KerasClassifier(
        build_fn=create_lstm_model,
        vocab_size=10000,
        input_length=100,
        num_classes=3,
        epochs=10,
        batch_size=32,
        verbose=1)
    )
], memory=memory)


In [20]:

scores = cross_val_score(pipeline, X_transformed, y, cv=10, scoring="accuracy")


Data Shape Before LSTM: (3090, 100)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8035 - loss: 0.7081
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8462 - loss: 0.4844
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9259 - loss: 0.2130
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9749 - loss: 0.0913
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9939 - loss: 0.0307
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9983 - loss: 0.0090
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9986 - loss: 0.0065
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9913 - loss: 0.0337
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9986 - loss: 0.0060
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9991 - loss: 0.0062
Data Shape Before LSTM: (344, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Data Shape Before LSTM: (3090, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7866 - loss: 0.7182
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8449 - loss: 0.4573
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9185 - loss: 0.2194
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9777 - loss: 0.0789
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9903 - loss: 0.0456
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9964 - loss: 0.0178
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9963 - loss: 0.0160
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9984 - loss: 0.0082
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9993 - loss: 0.0048
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9991 - loss: 0.0036
Data Shape Before LSTM: (344, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Data Shape Before LSTM: (3090, 100)
Epoch 1/10


...

97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7958 - loss: 0.7062
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8368 - loss: 0.4674
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9034 - loss: 0.2497
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9579 - loss: 0.1208
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9909 - loss: 0.0359
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9979 - loss: 0.0127
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9991 - loss: 0.0083
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9969 - loss: 0.0141
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9974 - loss: 0.0082
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9969 - loss: 0.0139
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Data Shape Before LSTM: (3091, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8058 - loss: 0.7131
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8448 - loss: 0.4658
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9069 - loss: 0.2244
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9737 - loss: 0.1032
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9925 - loss: 0.0432
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9970 - loss: 0.0166
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9961 - loss: 0.0189
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9978 - loss: 0.0083
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9992 - loss: 0.0058
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9979 - loss: 0.0068
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Data Shape Before LSTM: (3091, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8628 - loss: 0.5581
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8945 - loss: 0.3647
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9443 - loss: 0.1773
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9647 - loss: 0.1038
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9945 - loss: 0.0284
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9979 - loss: 0.0142
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9982 - loss: 0.0074
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9966 - loss: 0.0162
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9982 - loss: 0.0055
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9989 - loss: 0.0042
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [21]:
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {np.mean(scores):.2f}")


Cross-validation scores: [0.85174419 0.93895349 0.76162791 0.69476744 0.72886297 0.82507289
 0.90379009 0.88629738 0.82507289 0.32069971]
Mean accuracy: 0.77


In [23]:
# for classification tasks, prefer StratifiedKFold to maintain class balance, instead of TimeSeriesSplit
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [33]:


def create_lstm_model_sparse(vocab_size: int, input_length: int, num_classes: int):
    model = Sequential([
        Input(shape=(input_length,), name="input_layer"),  # explicit input layer
        Embedding(input_dim=vocab_size, output_dim=128, name="embedding_layer"),
        LSTM(128, return_sequences=True, name="lstm_layer_1"),
        Dropout(0.2, name="dropout_layer_1"),
        LSTM(64, name="lstm_layer_2"),
        Dropout(0.2, name="dropout_layer_2"),
        Dense(num_classes, activation="softmax", name="output_layer")
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# set up caching directory
memory_sparse = Memory(location="cache_directory_sparse", verbose=0)

pipeline_sparse = Pipeline([
    # ("keyphrase_extractor", # don't do this!, pre-compute this step instead!!!
    #   KeyphraseExtractor(nfeatures=10000, doclen=100)),
    ('debug', DebugTransformer()),
    ("lstm_classifier",
      KerasClassifier(
        build_fn=create_lstm_model_sparse,
        vocab_size=10000,
        input_length=100,
        num_classes=3,
        epochs=10,
        batch_size=32,
        verbose=1)
    )
], memory=memory_sparse)


In [36]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_indices = label_encoder.fit_transform(rate_speeches["Direction"])  # integer encoding
display(y_indices.shape)

(3434,)

In [37]:

scores = cross_val_score(pipeline_sparse, X_transformed, y_indices, cv=skf, scoring="accuracy")


Data Shape Before LSTM: (3090, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8115 - loss: 0.6879
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8554 - loss: 0.4500
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9210 - loss: 0.2238
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9785 - loss: 0.0770
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9891 - loss: 0.0504
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9938 - loss: 0.0193
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9966 - loss: 0.0201
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9971 - loss: 0.0088
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9990 - loss: 0.0053
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9980 - loss: 0.0060
Data Shape Before LSTM: (344, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Data Shape Before LSTM: (3090, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8097 - loss: 0.6824
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8536 - loss: 0.4483
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9169 - loss: 0.2250
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9710 - loss: 0.1055
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9897 - loss: 0.0417
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9942 - loss: 0.0279
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9985 - loss: 0.0117
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9990 - loss: 0.0065
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9975 - loss: 0.0184
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9982 - loss: 0.0075
Data Shape Before LSTM: (344, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Data Shape Before LSTM: (3090, 100)
Epoch 1/10


...

97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8041 - loss: 0.6763
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8578 - loss: 0.4297
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9184 - loss: 0.2158
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9570 - loss: 0.1297
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9916 - loss: 0.0404
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9949 - loss: 0.0306
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9962 - loss: 0.0284
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9980 - loss: 0.0147
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9931 - loss: 0.0231
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9976 - loss: 0.0112
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Data Shape Before LSTM: (3091, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8169 - loss: 0.6755
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8402 - loss: 0.4793
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9112 - loss: 0.2141
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9728 - loss: 0.1006
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9914 - loss: 0.0377
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9982 - loss: 0.0112
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9982 - loss: 0.0104
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9977 - loss: 0.0090
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9990 - loss: 0.0053
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9998 - loss: 0.0015
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Data Shape Before LSTM: (3091, 100)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8053 - loss: 0.6829
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8417 - loss: 0.4766
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9270 - loss: 0.2135
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9803 - loss: 0.0794
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9944 - loss: 0.0299
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9937 - loss: 0.0233
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9972 - loss: 0.0211
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9978 - loss: 0.0136
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9932 - loss: 0.0238
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9985 - loss: 0.0084
Data Shape Before LSTM: (343, 100)
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [38]:
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {np.mean(scores):.2f}")


Cross-validation scores: [0.80523256 0.81395349 0.7994186  0.78488372 0.79591837 0.79591837
 0.8425656  0.83381924 0.83090379 0.82215743]
Mean accuracy: 0.81
